In [1]:
import serial
import time
import binascii
from struct import *
class CySmart:
    Commands = {
        'CMD_Resolve_and_Set_Peer_Device_BD_Address':binascii.unhexlify("A1FE"),
        'CMD_Hedder':binascii.unhexlify("4359"),
        'CMD_Footer':binascii.unhexlify("0000"),
        'CMD_INIT_BLE_STACK':binascii.unhexlify("07FC"),
        'CMD_START_SCAN':binascii.unhexlify("93FE"),
        'CMD_STOP_SCAN':binascii.unhexlify("94FE"),
        'CMD_ESTABLISH_CONNECTION':binascii.unhexlify("97FE"),
        'CMD_TERMINATE_CONNECTION':binascii.unhexlify("98FE"),
        'CMD_EXCHANGE_GATT_MTU_SIZE':binascii.unhexlify("12FE"),
        'CMD_READ_CHARACTERISTIC_VALUE':binascii.unhexlify("06FE"),
        'CMD_READ_USING_CHARACTERISTIC_UUID':binascii.unhexlify("07FE"),Flag_Exchange_RETURN
        'CMD_WRITE_CHARACTERISTIC_VALUE':binascii.unhexlify("0BFE"),
        'CMD_WRITE_CHARACTERISTIC_VALUE_WITHOUT_RESPONSE':binascii.unhexlify("0AFE"),
        'CMD_FIND_INCLUDED_SERVICES':binascii.unhexlify("02FE"),
        'CMD_DISCOVER_ALL_CHARACTERISTICS':binascii.unhexlify("03FE"),
        'CMD_INITIATE_PAIRING_REQUEST':binascii.unhexlify("99FE"),
        'CMD_UPDATE_CONNECTION_PARAMETER_RESPONSE':binascii.unhexlify("9FFE")
    }
    
    
    
    Flag_DISABLE_ALL_CHECK = 0x00
    Flag_CHECK_PARAMETER_LENGTH = 0x1
    Flag_IMMEDIATE_RESPONSE = 0x2
    Flag_API_RETURN = 0x4
    Flag_Exchange_RETURN = 0x3
    Flag_TRIGGER_COMPLETE = 0x8
    Flag_SECONDARY_CMD = 0x10
    
    CYSMT_EVT_HEADER_CODE = binascii.unhexlify("BDA7")
    EVT_SCAN_PROGRESS_RESULT = binascii.unhexlify("8A06")
    EVT_COMMAND_STATUS =  binascii.unhexlify("7E04")
    EVT_COMMAND_COMPLETE =  binascii.unhexlify("7F04")
    EVT_READ_CHARACTERISTIC_VALUE_RESPONSE  =  binascii.unhexlify("0606")

    def __init__(self):
        pass

    def start(self,flag, ComPort='\\.\COM6' ):
        self.serin = serial.Serial(ComPort, 115200, timeout=3)
        self.serin.isOpen()
        self.Flag_RETURN = flag
        
        
                 
    def hexPrint(self,s):
        if type(s) is not int:
            return ":".join("{:02x}".format(ord(c)) for c in s)
        return "{:02x}".format(s)
    
    def hexArray(self,s):
        return self.hexPrint(s).split(":")
    
    def sendCommand(self,command, hedder = True, footer = True,whateforCompleate =True):
        cmd = ""
        if hedder:
            cmd += self.Commands['CMD_Hedder']
        cmd += command
        
        if footer:
            cmd+= self.Commands['CMD_Footer']
        
        self.serin.write(cmd)
        self.serin.flush()
        time.sleep(.1)
        #print "wating for output:"+self.hexPrint(command)
        return self.prossesOutput(whateforCompleate)
    
    
    def prossesOutput(self,whateforCompleate):
        payloads = {}
        time.sleep(.1)
        timoutTime = 3
        while 1 :
            while self.serin.inWaiting() < 2 and timout < timoutTime:
                time.sleep(.05)
                timout +=.05
            
            if(self.serin.inWaiting() < 2 and timout >= timoutTime):
                return payloads
            else:
                timout = 0
            
            hedderTest = self.serin.read(2)
           
            if self.CYSMT_EVT_HEADER_CODE in hedderTest:
                #Have Hedder message
                mesageLen = self.serin.read(2)
                mesageLen = unpack('h',mesageLen)[0]
                message = self.serin.read(mesageLen)
            
                if len(message) > 4:
                    #print "have message"
                    
                    #print self.hexArray(message) 
                    
                    event, command = unpack('2s2s', message[0:4])
                    
                            
                    if event not in self.EVT_COMMAND_STATUS and event not in self.EVT_COMMAND_COMPLETE:
                        body = message[4:]
                        if event not in payloads:
                             payloads[event] = []      
                        payloads[event].append(body)
                        
                        
                    if whateforCompleate == True:
                       
                        if event in self.EVT_COMMAND_COMPLETE:
                            #print "Compleat"
                            return payloads
                    else:
                        #print "Dont whateforCompleate"  
                        if event not in self.EVT_COMMAND_STATUS:
                             return payloads
        return payloads
        
    def getScanData(self,cyd):
        Ble = {'address':[], 'name':""}
        
        if self.EVT_SCAN_PROGRESS_RESULT in cyd:
            for scan in cyd[self.EVT_SCAN_PROGRESS_RESULT]:
                if len(scan) > 10:
                    inputString = scan
                    Ble['address'] = inputString[1:6]
                    if '\t' in inputString:
                        nm_length=  int(self.hexArray(inputString.split('\t')[0])[-1],16)-1
                        Ble['name'] = inputString.split('\t')[1][0:nm_length]
        return Ble
    
    
    def openConection(self,address):
        out = {'CMD_Resolve_and_Set_Peer_Device_BD_Address':{},
               'CMD_ESTABLISH_CONNECTION':{},
               'EXCHANGE_GATT_MTU_SIZE':{},
               'Read_using_Characteristic_UUID':{}
               }
        
        cmd = self.Commands['CMD_Resolve_and_Set_Peer_Device_BD_Address']+ binascii.unhexlify("0700") +address 
        out['CMD_Resolve_and_Set_Peer_Device_BD_Address'] = self.sendCommand(cmd)
        
        cmd = self.Commands['CMD_ESTABLISH_CONNECTION'] + binascii.unhexlify("0700") +address
        out['CMD_ESTABLISH_CONNECTION'] = self.sendCommand(cmd)
        
        out['EXCHANGE_GATT_MTU_SIZE'] = self.EXCHANGE_GATT_MTU_SIZE(0x0200)
        
        out['Read_using_Characteristic_UUID'] = self.Read_using_Characteristic_UUID(0x0001,0xFFFF,0x2A00)
        return out
        
    def close_Conection(self):
        cmd = self.Commands['CMD_TERMINATE_CONNECTION']
        cmd += binascii.unhexlify("02000400")
        return self.sendCommand(cmd)
    
    def _RETURN(self,pack,prams):
        values = (self.Flag_RETURN,)
        if type(prams) == tuple and type(pack) == str:
             values += prams
                
        pack = '=H '+ pack
        s = Struct(pack)
        packed_data = s.pack(*values)
        h=Struct('H')
        packsize = h.pack(s.size) 
        packed_data = packsize + packed_data
        return packed_data
    
    def EXCHANGE_GATT_MTU_SIZE(self,size):
        cmd = self.Commands['CMD_EXCHANGE_GATT_MTU_SIZE']
        
        cmd += self._RETURN('H',(0x200,))
        print self.hexArray(cmd)
        return self.sendCommand(cmd,footer=False)
    
    def Read_using_Characteristic_UUID(self,Start_Handle,End_Handle,UUID):
        cmd = self.Commands['CMD_READ_USING_CHARACTERISTIC_UUID']
        #cmd += self._RETURN('H H H H',(0x100, UUID, Start_Handle, End_Handle))
        cmd += self._RETURN('B H H H',(0x01, UUID, Start_Handle, End_Handle))
        return self.sendCommand(cmd,footer=False)
    
    def Read_Characteristic_Value(self,Attribute):
        cmd = self.Commands['CMD_READ_CHARACTERISTIC_VALUE']
        cmd += pack('H H H',*(self.Flag_RETURN, ReturnFlag, Attribute ))
        Response = self.sendCommand(cmd,footer=False)
        print"Read_Characteristic_Value: ",Response
        #event, rest = Response[0:3], Response[4:]
        out_Response = []
        if self.EVT_READ_CHARACTERISTIC_VALUE_RESPONSE in Response:
                for cs in  Response[self.EVT_READ_CHARACTERISTIC_VALUE_RESPONSE]:
                    out_Response.append(cs[4:])
                    
        return out_Response
    
    def Write_Characteristic_Value(self,Attribute,payload):
        pramcount = binascii.unhexlify("0300")
        le = len(payload)
        package = pramcount+pack("H",*(Attribute,))+pack("H",*(le,))+payload
        package = pack("H",*(len(package),)) + package
        cmd = self.Commands['CMD_WRITE_CHARACTERISTIC_VALUE']
        cmd += package
        return self.sendCommand(cmd,footer=False)
    
    def Read_All_characteristics(self,data_set):
        for se in data_set:
            data_set[se] = self.Read_Characteristic_Value(se)
        return data_set
    
    def Initiate_Pairing(self):
        cmd = self.Commands['CMD_INITIATE_PAIRING_REQUEST']
        cmd += pack('H H',*(self.Flag_IMMEDIATE_RESPONSE, self.Flag_RETURN ))
        return self.sendCommand(cmd,footer=False)
    
    def Update_Connection_Parameter(self,Response):
        cmd = self.Commands['CMD_UPDATE_CONNECTION_PARAMETER_RESPONSE']
        if Response:
            cmd += binascii.unhexlify("040003000000")
        else:
            cmd += binascii.unhexlify("040003000100")
            
        return self.sendCommand(cmd,footer=False)
    
    def close(self):
        self.serin.close()
        
        

cy = CySmart()



In [3]:

#cy.start(cy.Flag_API_RETURN)
cy.start(cy.Flag_Exchange_RETURN)

cy.sendCommand(cy.Commands['CMD_INIT_BLE_STACK'])
cyd= cy.sendCommand(cy.Commands['CMD_START_SCAN'],whateforCompleate=False)
time.sleep(5)
cy.sendCommand(cy.Commands['CMD_STOP_SCAN'])

if cy.EVT_SCAN_PROGRESS_RESULT in cyd:
    client = cy.getScanData(cyd)
    print client['name'] , cy.hexArray(client['address'])
    print cy.openConection(client['address'])
    #print cy.Update_Connection_Parameter(False)
    #cy.Initiate_Pairing()
        

    #print cy.Read_Characteristic_Value(0x000E)
    #print cy.Write_Characteristic_Value(0x000E,binascii.unhexlify("01020304"))
    #print cy.Read_Characteristic_Value(0x000E)
    cy.close_Conection()
else:
     print "nothing found:"
cy.close()


SerialException: could not open port 'COM4': WindowsError(2, 'The system cannot find the file specified.')

In [6]:

    data_set = {0x0003:{},
                    0x0005:{},
                    0x0007:{},
                    0x000A:{},
                    0x000E:{},
                    0x0010:{},
                    0x0014:{},
                    0x0018:{},
                    0x001D:{},
                    0x001F:{},
                    0x0022:{}
                    }
    
    allcs =  cy.Read_All_characteristics(data_set)
    for cs in allcs:
        print cy.hexPrint(cs), allcs[cs]

IndentationError: unexpected indent (<ipython-input-6-679d4f252d9b>, line 2)

In [14]:
test = ['1d', '2a', '05', '50', 'a0']
print

1d2a0550a0


In [19]:


s=Struct('H H H')
packed_data= s.pack(*(cy.Flag_API_RETURN,cy.Flag_API_RETURN,0x0007))

print 'Original values:', cy.Flag_API_RETURN
print 'Format string  :', s.format
print 'Uses           :', s.size, 'bytes'
print 'Packed Value   :', binascii.hexlify(packed_data)
#packed_data =  pack('=H',s.size)  + packed_data
print binascii.hexlify(packed_data)



Original values: 4
Format string  : H H H
Uses           : 6 bytes
Packed Value   : 040004000700
040004000700


In [21]:
data_set = {0x0003:{},
                    0x0005:{},
                    0x0007:{},
                    0x000A:{},
                    0x000E:{},
                    0x0010:{},
                    0x0014:{},
                    0x0018:{},
                    0x001D:{},
                    0x001F:{},
                    0x0022:{}
                    }
for se in data_set:
    print se
    data_set[se] = {1:'test'}
print data_set

3
5
7
10
14
16
20
24
29
31
34
{3: {1: 'test'}, 5: {1: 'test'}, 7: {1: 'test'}, 10: {1: 'test'}, 14: {1: 'test'}, 16: {1: 'test'}, 20: {1: 'test'}, 24: {1: 'test'}, 29: {1: 'test'}, 31: {1: 'test'}, 34: {1: 'test'}}


In [19]:
import binascii
from struct import *

#4359 0BFE 0A00 0300 0E00 0400 01020304
#               0300 0e00 0400 01020304


print binascii.hexlify(package)



0a0003000e00040001020304
